In [2]:
%run Imports.ipynb

conf = pyspark.SparkConf()
spark = SparkSession.builder.appName("IngestDune").config(conf=conf).getOrCreate()

/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
23/03/25 17:58:59 WARN Utils: Your hostname, Yashasvis-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.232 instead (on interface en0)
23/03/25 17:58:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/25 17:59:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
dune_query = {'90': 2201677, '91': 2203423, '92': 2203427, '93': 2203429,'94': 2203430,'95': 2203432,'96': 2203524,'97': 2203528,'98': 2203530,
                  '99': 2203531,'d0': 2203532,'d1': 2203534,'d2': 2203571,'d3': 2203573,'d4': 2203575,'d5': 2203576,'d6': 2203577,'d7': 2203580,
                 'd8': 2203581,'d9': 2203582,'e0': 2203608,'e1': 2203614,'e2': 2203616,'e3': 2203617,'e4': 2203618,'e5': 2203620,'e6': 2203622,
                 'e7': 2203623,'e8': 2203624,'e9': 2203625,'f0': 2203758,'f1': 2203761,'f2': 2203762,'f3': 2203767,'f4': 2203770,'f5': 2203771,
                 'f6': 2203772,'f7': 2203774,'f8': 2203775,'f9': 2203777, }

In [5]:
query_list_a = list(dune_query.values())

In [6]:
query_list_y = [2202508, 2202546, 2202652, 2202655, 2202656, 2202657, 2202744, 2202751, 2202756, 2202758, 2202760, 2202762, 2202764, 2202813, 2202816, 2202820,2202850, 2202851, 2202875, 2202879, 2202880, 2202882, 2202883, 2202885, 2202886, 2202891, 2202892, 2202893, 2202896, 2202900, 2202907, 2202914,2202920, 2202921, 2202923, 2202924, 2202929, 2202931, 2202934, 2202936, 2202937, 2202939, 2202940, 2202962, 2202964, 2202966, 2202967, 2202970,2202985, 2202986, 2202987, 2202988, 2202991, 2203002, 2203004, 2203007, 2203011, 2203014, 2203021, 2203023, 2203025, 2203028, 2203034, 2203036]

In [7]:
query_list_p = [2202609,2202623,2202627,2202634,2202649,2202660,2202665,2202680,2202712,2202722,2202733,2202737,2202747,2202750,2202761,2202766,2202632,2202645,2202654,2202658,2202671,2202677,2202679,2202683,2202686,2202689,2202692,2202705,2202709,2202711,2202724,2202725,2202736,2202738,2202740,2202745,2202746,2202748,2202754,2202759,2202763,2202778,2202781,2202785,2202792,2202796,2202798,2202799,2202768,2202780,2202783,2202784,2202787,2202791,2202800,2202802,2202804,2202809,2202815,2202819,2202822,2202810,2202814,2202817]

In [37]:
query_list_m = [2174066, 2167380, 2203067, 2203087,  2203123, 203198, 2203199, 2203204,  2203242,  2203358, 2203364, 2203384, 2203358, 2203392, 2203415,  2203418, 2203419, 2203421, 2203589, 2203621 ,2203637 , 2203644  , 2207729 , 2207754 , 2207790  , 207797  , 2207812  , 2207847 , 2207865 , 2207878, 2207905, 2207914, 2207921, 2232809, 2232830, 2232838, 2232855, 2232872, 2232889, 2232911, 2258257, 2258262, 2258274, 2258291, 2258322, 2258389, 2258402, 2258433, 2260264, 2260533, 2260731, 2260783, 2260862, 2260902, 2261035, 2261073, 2261179, 2261823, 2261824, 2261825, 2261827, 2261828, 2261829, 2261834]

In [9]:
def combine_df(df):
    combined_df_all = pd.concat(df, ignore_index=True)
    return combined_df_all

In [71]:
def get_final_list(pd_list):
    pd_list_new = []
    for pdn in pd_list:
        df_new = pdn[['block_number','block_time','buyer', 'category', 'evt_type', 'nft_contract_address', 'nft_contract_addresses_array',
                   'nft_project_name', 'nft_token_id', 'nft_token_ids_array', 'number_of_items', 'original_amount', 'original_currency',
                   'platform', 'seller', 'trade_id', 'trade_type', 'usd_amount']]
        pd_list_new.append(df_new)
    return pd_list_new

In [72]:
def get_data_dune(query_list):
    pd_list = []
    for query in tqdm(query_list):
        pd_name = dune.refresh_into_dataframe(Query(query_id=query,))
        pd_name['block_time'] =  pd.to_datetime(pd_name['block_time'], infer_datetime_format=True)
        pd_list.append(pd_name)
    return pd_list

In [73]:
def get_final_data(extracted_raw_data_y, query_list_y, varname):
    for idx, df in enumerate(extracted_raw_data_y):
        df = df.replace('<nil>', np.nan)
        df = df.dropna()
        rows_with_nil = df[df.isin(['<nil>']).any(axis=1)]
    for i, df in enumerate(extracted_raw_data_y):
        df['original_amount'] = df['original_amount'].replace('<nil>', 0.0).astype(float)
        df['usd_amount'] = df['usd_amount'].replace('<nil>', 0.0).astype(float)
        df['number_of_items'] = df['number_of_items'].replace('<nil>', 0).astype(int)
        df['nft_token_id'] = df['nft_token_id'].astype(str)
    for i, df in enumerate(extracted_raw_data_y):
        table = pa.Table.from_pandas(df)
        output_file = f"/Users/yashasvi/Documents/loadDune_{varname}/output_{query_list_y[i]}.parquet"
        pq.write_table(table, output_file)

In [74]:
def entireData(query_list_parts, varname):
    for i in range(len(query_list_parts)):
        original_raw_data = get_data_dune(list(query_list_parts[i]))
        extracted_raw_data = get_final_list(original_raw_data)
        get_final_data(extracted_raw_data, query_list_parts[i], varname)

In [75]:
query_list_y_parts = np.array_split(query_list_y, 8)
entireData(query_list_y_parts, "part1")

  0%|                                                     | 0/8 [00:00<?, ?it/s]2023-03-25 20:55:24,439 INFO dune_client.base_client waiting for query execution 01GWDPKC42J89ACHXE90Q7FH3A to complete: ExecutionState.EXECUTING
2023-03-25 20:55:29,839 INFO dune_client.base_client waiting for query execution 01GWDPKC42J89ACHXE90Q7FH3A to complete: ExecutionState.EXECUTING
2023-03-25 20:55:35,223 INFO dune_client.base_client waiting for query execution 01GWDPKC42J89ACHXE90Q7FH3A to complete: ExecutionState.EXECUTING
2023-03-25 20:55:40,619 INFO dune_client.base_client waiting for query execution 01GWDPKC42J89ACHXE90Q7FH3A to complete: ExecutionState.EXECUTING
2023-03-25 20:55:46,014 INFO dune_client.base_client waiting for query execution 01GWDPKC42J89ACHXE90Q7FH3A to complete: ExecutionState.EXECUTING
2023-03-25 20:55:51,420 INFO dune_client.base_client waiting for query execution 01GWDPKC42J89ACHXE90Q7FH3A to complete: ExecutionState.EXECUTING
2023-03-25 20:55:56,805 INFO dune_client.bas

  0%|                                                     | 0/8 [00:00<?, ?it/s]2023-03-25 20:58:38,484 INFO dune_client.base_client waiting for query execution 01GWDPS9KW3YAPANYWGQWQPPVP to complete: ExecutionState.EXECUTING
2023-03-25 20:58:43,849 INFO dune_client.base_client waiting for query execution 01GWDPS9KW3YAPANYWGQWQPPVP to complete: ExecutionState.EXECUTING
/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/dune_client/client.py:182: DtypeWarning: Columns (10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(data)
 12%|█████▋                                       | 1/8 [00:17<02:00, 17.25s/it]2023-03-25 20:58:55,678 INFO dune_client.base_client waiting for query execution 01GWDPSTE27X886A0WPSX54BJ4 to complete: ExecutionState.EXECUTING
2023-03-25 20:59:01,073 INFO dune_client.base_client waiting for query execution 01GWDPSTE27X886A0WPSX54BJ4 to complete: ExecutionState.EXECUTING
/Users/yashasvi/o

  0%|                                                     | 0/8 [00:00<?, ?it/s]2023-03-25 21:01:58,201 INFO dune_client.base_client waiting for query execution 01GWDPZCNRRQ6M4S09FDWB6D7M to complete: ExecutionState.EXECUTING
2023-03-25 21:02:03,608 INFO dune_client.base_client waiting for query execution 01GWDPZCNRRQ6M4S09FDWB6D7M to complete: ExecutionState.EXECUTING
/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/dune_client/client.py:182: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(data)
 12%|█████▋                                       | 1/8 [00:15<01:46, 15.19s/it]2023-03-25 21:02:13,391 INFO dune_client.base_client waiting for query execution 01GWDPZVFTPENSVZ4GNMRQPRRN to complete: ExecutionState.EXECUTING
2023-03-25 21:02:18,832 INFO dune_client.base_client waiting for query execution 01GWDPZVFTPENSVZ4GNMRQPRRN to complete: ExecutionState.EXECUTING
/Users/yashasvi/opt/ana

 25%|███████████▎                                 | 2/8 [00:35<01:45, 17.64s/it]2023-03-25 21:05:02,089 INFO dune_client.base_client waiting for query execution 01GWDQ50871J4NFNGWQPEAHR3Y to complete: ExecutionState.EXECUTING
2023-03-25 21:05:07,492 INFO dune_client.base_client waiting for query execution 01GWDQ50871J4NFNGWQPEAHR3Y to complete: ExecutionState.EXECUTING
/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/dune_client/client.py:182: DtypeWarning: Columns (7,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(data)
 38%|████████████████▉                            | 3/8 [00:52<01:27, 17.47s/it]2023-03-25 21:05:19,369 INFO dune_client.base_client waiting for query execution 01GWDQ5H3R3MZNH72VQRWXYYKF to complete: ExecutionState.EXECUTING
2023-03-25 21:05:24,870 INFO dune_client.base_client waiting for query execution 01GWDQ5H3R3MZNH72VQRWXYYKF to complete: ExecutionState.EXECUTING
/Users/yashasvi

/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/dune_client/client.py:182: DtypeWarning: Columns (2,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(data)
 62%|████████████████████████████▏                | 5/8 [01:22<00:49, 16.54s/it]2023-03-25 21:08:02,229 INFO dune_client.base_client waiting for query execution 01GWDQAG5GPRVC8A0YRFZR4S9H to complete: ExecutionState.EXECUTING
2023-03-25 21:08:07,618 INFO dune_client.base_client waiting for query execution 01GWDQAG5GPRVC8A0YRFZR4S9H to complete: ExecutionState.EXECUTING
/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/dune_client/client.py:182: DtypeWarning: Columns (2,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(data)
 75%|█████████████████████████████████▊           | 6/8 [01:38<00:32, 16.24s/it]2023-03-25 21:08:17,940 INFO dune_client.base_client waiting for q

 88%|███████████████████████████████████████▍     | 7/8 [01:51<00:15, 15.99s/it]2023-03-25 21:10:43,426 INFO dune_client.base_client waiting for query execution 01GWDQFDJM8R02ACFSHFQR6441 to complete: ExecutionState.EXECUTING
2023-03-25 21:10:48,810 INFO dune_client.base_client waiting for query execution 01GWDQFDJM8R02ACFSHFQR6441 to complete: ExecutionState.EXECUTING
2023-03-25 21:10:54,188 INFO dune_client.base_client waiting for query execution 01GWDQFDJM8R02ACFSHFQR6441 to complete: ExecutionState.EXECUTING
2023-03-25 21:10:59,560 INFO dune_client.base_client waiting for query execution 01GWDQFDJM8R02ACFSHFQR6441 to complete: ExecutionState.EXECUTING
2023-03-25 21:11:04,924 INFO dune_client.base_client waiting for query execution 01GWDQFDJM8R02ACFSHFQR6441 to complete: ExecutionState.EXECUTING
2023-03-25 21:11:10,325 INFO dune_client.base_client waiting for query execution 01GWDQFDJM8R02ACFSHFQR6441 to complete: ExecutionState.EXECUTING
2023-03-25 21:11:15,728 INFO dune_client.bas

2023-03-25 21:13:59,288 INFO dune_client.base_client waiting for query execution 01GWDQMX18V08YZXB5VHBFTXMS to complete: ExecutionState.EXECUTING
2023-03-25 21:14:04,687 INFO dune_client.base_client waiting for query execution 01GWDQMX18V08YZXB5VHBFTXMS to complete: ExecutionState.EXECUTING
2023-03-25 21:14:10,126 INFO dune_client.base_client waiting for query execution 01GWDQMX18V08YZXB5VHBFTXMS to complete: ExecutionState.EXECUTING
2023-03-25 21:14:15,522 INFO dune_client.base_client waiting for query execution 01GWDQMX18V08YZXB5VHBFTXMS to complete: ExecutionState.EXECUTING
/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/dune_client/client.py:182: DtypeWarning: Columns (7,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(data)
 75%|█████████████████████████████████▊           | 6/8 [02:52<01:06, 33.22s/it]2023-03-25 21:14:27,424 INFO dune_client.base_client waiting for query execution 01GWDQP89WPGPE

 75%|█████████████████████████████████▊           | 6/8 [01:56<00:35, 17.70s/it]2023-03-25 21:17:14,269 INFO dune_client.base_client waiting for query execution 01GWDQVB7Z51G7DCA3FXDWWD96 to complete: ExecutionState.EXECUTING
2023-03-25 21:17:19,665 INFO dune_client.base_client waiting for query execution 01GWDQVB7Z51G7DCA3FXDWWD96 to complete: ExecutionState.EXECUTING
 88%|███████████████████████████████████████▍     | 7/8 [02:10<00:16, 16.62s/it]2023-03-25 21:17:28,648 INFO dune_client.base_client waiting for query execution 01GWDQVS9N7H73GWSPDRBHBBF7 to complete: ExecutionState.EXECUTING
2023-03-25 21:17:34,044 INFO dune_client.base_client waiting for query execution 01GWDQVS9N7H73GWSPDRBHBBF7 to complete: ExecutionState.EXECUTING
/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/dune_client/client.py:182: DtypeWarning: Columns (2,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(data)
100%|██████████

In [76]:
query_list_p_parts = np.array_split(query_list_p, 8)
entireData(query_list_p_parts, "part2")

  0%|                                                     | 0/8 [00:00<?, ?it/s]2023-03-25 21:21:17,631 INFO dune_client.base_client waiting for query execution 01GWDR2RX131EKKS1BX3365Y1C to complete: ExecutionState.EXECUTING
2023-03-25 21:21:23,032 INFO dune_client.base_client waiting for query execution 01GWDR2RX131EKKS1BX3365Y1C to complete: ExecutionState.EXECUTING
/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/dune_client/client.py:182: DtypeWarning: Columns (10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(data)
 12%|█████▋                                       | 1/8 [00:16<01:55, 16.46s/it]2023-03-25 21:21:34,033 INFO dune_client.base_client waiting for query execution 01GWDR38WP2TH5HKCQ17FVN0JM to complete: ExecutionState.EXECUTING
2023-03-25 21:21:39,412 INFO dune_client.base_client waiting for query execution 01GWDR38WP2TH5HKCQ17FVN0JM to complete: ExecutionState.EXECUTING
/Users/yashasvi/o

2023-03-25 21:24:37,623 INFO dune_client.base_client waiting for query execution 01GWDR8PYP5D279VT2QH1K7XG9 to complete: ExecutionState.EXECUTING
/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/dune_client/client.py:182: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(data)
 12%|█████▋                                       | 1/8 [00:13<01:35, 13.65s/it]2023-03-25 21:24:45,912 INFO dune_client.base_client waiting for query execution 01GWDR949F75ZJBGGNNW03DKBZ to complete: ExecutionState.EXECUTING
2023-03-25 21:24:51,302 INFO dune_client.base_client waiting for query execution 01GWDR949F75ZJBGGNNW03DKBZ to complete: ExecutionState.EXECUTING
/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/dune_client/client.py:182: DtypeWarning: Columns (2,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(data)
 25%|██████

 38%|████████████████▉                            | 3/8 [00:45<01:15, 15.12s/it]2023-03-25 21:27:35,705 INFO dune_client.base_client waiting for query execution 01GWDREA3E0WJNBDZT7RKKGBYX to complete: ExecutionState.EXECUTING
2023-03-25 21:27:41,149 INFO dune_client.base_client waiting for query execution 01GWDREA3E0WJNBDZT7RKKGBYX to complete: ExecutionState.EXECUTING
/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/dune_client/client.py:182: DtypeWarning: Columns (10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(data)
 50%|██████████████████████▌                      | 4/8 [00:59<00:59, 14.85s/it]2023-03-25 21:27:50,127 INFO dune_client.base_client waiting for query execution 01GWDRER6DKDT7R0RC5JD6EP9V to complete: ExecutionState.EXECUTING
2023-03-25 21:27:55,538 INFO dune_client.base_client waiting for query execution 01GWDRER6DKDT7R0RC5JD6EP9V to complete: ExecutionState.EXECUTING
/Users/yashasvi/o

/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/dune_client/client.py:182: DtypeWarning: Columns (10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(data)
 62%|████████████████████████████▏                | 5/8 [01:30<00:54, 18.16s/it]2023-03-25 21:30:51,842 INFO dune_client.base_client waiting for query execution 01GWDRM9NHJ46K1R4GB1WQKW44 to complete: ExecutionState.EXECUTING
2023-03-25 21:30:57,251 INFO dune_client.base_client waiting for query execution 01GWDRM9NHJ46K1R4GB1WQKW44 to complete: ExecutionState.EXECUTING
2023-03-25 21:31:02,657 INFO dune_client.base_client waiting for query execution 01GWDRM9NHJ46K1R4GB1WQKW44 to complete: ExecutionState.EXECUTING
2023-03-25 21:31:08,055 INFO dune_client.base_client waiting for query execution 01GWDRM9NHJ46K1R4GB1WQKW44 to complete: ExecutionState.EXECUTING
2023-03-25 21:31:13,469 INFO dune_client.base_client waiting for query execution 01GWDRM9NHJ46K1R

 38%|████████████████▉                            | 3/8 [01:44<03:07, 37.42s/it]2023-03-25 21:34:07,035 INFO dune_client.base_client waiting for query execution 01GWDRT8983YSMYK3ZH4YAMEHC to complete: ExecutionState.EXECUTING
2023-03-25 21:34:12,406 INFO dune_client.base_client waiting for query execution 01GWDRT8983YSMYK3ZH4YAMEHC to complete: ExecutionState.EXECUTING
/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/dune_client/client.py:182: DtypeWarning: Columns (10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(data)
 50%|██████████████████████▌                      | 4/8 [01:59<01:53, 28.38s/it]2023-03-25 21:34:21,568 INFO dune_client.base_client waiting for query execution 01GWDRTPF1SQEPJK8JQAMXG5VR to complete: ExecutionState.EXECUTING
2023-03-25 21:34:26,964 INFO dune_client.base_client waiting for query execution 01GWDRTPF1SQEPJK8JQAMXG5VR to complete: ExecutionState.EXECUTING
/Users/yashasvi/o

2023-03-25 21:40:36,238 INFO dune_client.base_client waiting for query execution 01GWDS5Z2GW4AGN5GT6B0H6R4H to complete: ExecutionState.EXECUTING
2023-03-25 21:40:41,672 INFO dune_client.base_client waiting for query execution 01GWDS5Z2GW4AGN5GT6B0H6R4H to complete: ExecutionState.EXECUTING
2023-03-25 21:40:47,072 INFO dune_client.base_client waiting for query execution 01GWDS5Z2GW4AGN5GT6B0H6R4H to complete: ExecutionState.EXECUTING
/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/dune_client/client.py:182: DtypeWarning: Columns (10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(data)
 38%|████████████████▉                            | 3/8 [01:31<02:37, 31.59s/it]2023-03-25 21:40:57,396 INFO dune_client.base_client waiting for query execution 01GWDS6S0AVNRW4RB5BBM1S373 to complete: ExecutionState.EXECUTING
2023-03-25 21:41:02,779 INFO dune_client.base_client waiting for query execution 01GWDS6S0AVNRW4R

2023-03-25 21:43:50,020 INFO dune_client.base_client waiting for query execution 01GWDSA7NQNCFG78B4T1AD7MRR to complete: ExecutionState.EXECUTING
/Users/yashasvi/opt/anaconda3/envs/itpcowdune/lib/python3.9/site-packages/dune_client/client.py:182: DtypeWarning: Columns (2,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(data)
 25%|███████████▎                                 | 2/8 [01:25<04:45, 47.65s/it]2023-03-25 21:44:00,481 INFO dune_client.base_client waiting for query execution 01GWDSCBT7Y6SR3J7BQJG3XD05 to complete: ExecutionState.EXECUTING
2023-03-25 21:44:05,860 INFO dune_client.base_client waiting for query execution 01GWDSCBT7Y6SR3J7BQJG3XD05 to complete: ExecutionState.EXECUTING
2023-03-25 21:44:11,254 INFO dune_client.base_client waiting for query execution 01GWDSCBT7Y6SR3J7BQJG3XD05 to complete: ExecutionState.EXECUTING
2023-03-25 21:44:16,646 INFO dune_client.base_client waiting for query execution 01GWDSCBT7Y6SR

In [ ]:
query_list_m_parts = np.array_split(query_list_m, 8)
entireData(query_list_m_parts, "part3")

  0%|                                                     | 0/8 [00:00<?, ?it/s]2023-03-25 21:50:26,792 INFO dune_client.base_client waiting for query execution 01GWDSR522ZBRD98D87Y2K89MC to complete: ExecutionState.EXECUTING
2023-03-25 21:50:32,169 INFO dune_client.base_client waiting for query execution 01GWDSR522ZBRD98D87Y2K89MC to complete: ExecutionState.EXECUTING
2023-03-25 21:50:37,563 INFO dune_client.base_client waiting for query execution 01GWDSR522ZBRD98D87Y2K89MC to complete: ExecutionState.EXECUTING


In [ ]:
# combine_df_y = combine_df(extracted_raw_data_y)

In [ ]:
# extracted_raw_data_y[0].dtypes

In [ ]:
# for i, df in enumerate(extracted_raw_data_y):
#     table = pa.Table.from_pandas(df)
#     output_file = f"/Users/yashasvi/Documents/loadDune/output_{query_list_y[i]}.parquet"
#     pq.write_table(table, output_file)
